In [ ]:
!pip install keras==2.3.1

In [ ]:
%tensorflow_version 2.x
import os
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, MaxPooling1D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, Input 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau 
from keras.callbacks import TensorBoard
from keras import backend as K 
from keras.optimizers import Adam, SGD, Adadelta
from keras.regularizers import l2, l1
import cv2
from keras.callbacks import CSVLogger
import sys
import time
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3

In [ ]:
training_data_directory = '/content/images/train'
validation_data_directory = '/content/images/test'
nb_train_samples = 67988 
nb_validation_samples = 22716
n_classes = 101
epochs = 10
batch_size = 75

In [ ]:
#Model that enable the freezing of the resnet layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005), 
                    activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

In [ ]:
#Model to enable the freezing of the resnet layers

base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))

X = base_model.output
X = AveragePooling2D(pool_size=(8, 8))(X)
X = Dropout(.4)(X)
X = Flatten()(X)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005),
                    activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# compile the model
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train & Test Data Generators with image augmentation 
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory(
    training_data_directory,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_data_generator.flow_from_directory(
    validation_data_directory,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# set up the checkpoint
checkpoint_filepath = './models/food-image-classification-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
logger = CSVLogger('./models/training.log')

In [ ]:
# reduce learning rate if validation loss does not improve after 3 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.1, 
                            patience=3, 
                            verbose=1, 
                            mode='auto', 
                            min_delta=0.0001,
                            cooldown=0, 
                            min_lr=0)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, logger, reduce_lr])


In [ ]:
# read log file
log_df = pd.read_csv('./models/training.log', sep=',', header=None)


In [ ]:
# Training and Test accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(x=log_df[0], y=log_df[1], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=log_df[0], y=log_df[2], name='Validation Accuracy'))
fig.update_layout(title='Training and Validation Accuracy', xaxis_title='Epochs', yaxis_title='Accuracy')
fig.show()

In [ ]:
# Training and Test loss
fig = go.Figure()
fig.add_trace(go.Scatter(x=log_df[0], y=log_df[3], name='Training Loss'))
fig.add_trace(go.Scatter(x=log_df[0], y=log_df[4], name='Validation Loss'))
fig.update_layout(title='Training and Validation Loss', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [ ]:
# model evaluation
score = model.evaluate(validation_generator, steps=nb_validation_samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
